In [ ]:
import random
import re
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter
import sys
from mnist import MNIST
mndata = MNIST('./2')

In [ ]:
pokoleniya=10000
population_size=100
chromosome_bodyoncetoldme=784 # длина хромосомы
selection_count=50 # сколько особей выбирается в селекции
mutation_kek=0.05 # шанс мутации, от 0 до 1
log=False # отображение данных после каждой операции

a=1 # скорость обучения
b=np.random.rand(1,10) # сдвиг
max_epochs=10 # эпохи обучения
print(b)

In [ ]:
def f(v): # функция активации
    if(v>0): return 1
    else: return 0

In [ ]:
images_train, labels_train = mndata.load_training()
train_fist=mndata.process_images_to_numpy(images_train)
train_ass=mndata.process_labels(labels_train)

In [ ]:
def predict(vector):
    result=0
    for i in range(10):
        if(vector[i]==1): result=i
    return result

# функция приспособленности
def prisposobl(w):
    shrek=np.vectorize(f)
    x=train_fist
    y=train_ass
    y1=x@w
    y1=shrek(y1)
    right=0
    for i in range(len(x)):
        if(predict(y1[i])==y[i]): right+=1

    accyracy=right/10000
    return accyracy # сколько правильных ответов

prisposobl(optimal)

In [ ]:
def generate_chrom(chrom_length):
    return

def generate_population(size):
    result=[]
    for i in range(size):
        temp=optimal[1:len(optimal)-1]
        random.shuffle(temp)
        temp.insert(0, optimal[0])
        temp.append(optimal[len(optimal)-1])
        result.append(temp)
    return result

generate_population(5)

In [ ]:
def print_population(data):
    df=pd.DataFrame(data)
    print(df)

# лучший маршрут в популяции
def population_best_result(data):
    result=999999
    best_shrek=[]
    for i in data:
        shrek=prisposobl(i)
        if(shrek<result): 
            result=shrek
            best_shrek=i
    return (result, best_shrek)

# худший маршрут в популяции
def population_max_result(data):
    penis=[]
    for i in data:
        penis.append((prisposobl(i), i))
    max_shrek=sorted(penis, key=itemgetter(0), reverse=True)[0]

    return (max_shrek)

# средний маршрут в популяции
def population_average_result(data):
    penis=[]
    for i in data:
        penis.append((prisposobl(i)))
    result=np.mean(penis)

    return result

# sex=generate_population(5)
# print(population_best_result(sex))
# print(population_max_result(sex))
# print(population_average_result(sex))

In [ ]:
# каловая селекция
def roulette_selection(data, sel_count):
    pool=[]
    sectors=[]
    wheel=[]
    f_sum=0

    for chromosome in data:
        f=1/(1+prisposobl(chromosome))
        f_sum+=f
    for chromosome in data:
        f=1/(1+prisposobl(chromosome))
        v=f/f_sum*100
        sectors.append((v, chromosome))
    # sectors=sorted(sectors, key=itemgetter(0))
    point=0
    for i in range(0,len(sectors)-1):
        wheel.append((sectors[i][1], (point, point+sectors[i][0])))
        point+=sectors[i][0]
        if(i==len(sectors)-2): 
            wheel.append((sectors[i][1], (point, 100)))
    # print(wheel)
    for i in range(sel_count):
        choice=random.randint(0,100)
        for j in wheel:
            # if((choice>=j[1][0]) and (choice<=j[1][1]) and (j[0] not in pool)): 
            if((choice>=j[1][0]) and (choice<=j[1][1])): 
                pool.append(j[0])
                break
    
    return pool

roulette_selection(generate_population(10), 5)

In [ ]:
# нормальная селекция
def simple_selection(data, sel_count):
    pool=[]
    adaptation=[]
    for chromosome in data:
        adaptation.append((prisposobl(chromosome), chromosome))
    adaptation=sorted(adaptation, key=itemgetter(0))
    for i in range(0, sel_count):
        pool.append(adaptation[i][1])

    return pool

simple_selection(generate_population(10), 5)

In [ ]:
# нормальное скрещивание
def permutation_crossover(parent1, parent2, chromo_length):
    chromo_length-=2
    par1_start=parent1[0]
    par2_start=parent2[0]
    par1_end=parent1[len(parent1)-1]
    par2_end=parent2[len(parent1)-1]
    parent1=parent1[1:len(parent1)-1]
    parent2=parent2[1:len(parent2)-1]

    pos1=random.randint(1, chromo_length//2-1)
    pos2=random.randint(pos1+1, chromo_length-1)
    middle=parent1[pos1:pos2+1]
    shrek=[]
    for i in parent2:
        if not (i in middle): shrek.append(i)
    child1=[]
    for i in range(0, chromo_length-1):
        if(i<pos1): child1.append(shrek.pop(0))
        elif(i==pos1): child1+=middle
        elif(i==pos2): child1.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)>0): child1.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)==0): child1.append(parent2[i])
    
    middle=parent2[pos1:pos2+1]
    shrek=[]
    for i in parent1:
        if not (i in middle): shrek.append(i)
    child2=[]
    for i in range(0, chromo_length-1):
        if(i<pos1): child2.append(shrek.pop(0))
        elif(i==pos1): child2+=middle
        elif(i==pos2): child2.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)>0): child2.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)==0): child2.append(parent1[i])

    child1.insert(0, par1_start)
    child1.append(par1_end)
    child2.insert(0, par2_start)
    child2.append(par2_end)
    return child1, child2

permutation_crossover([1,2,3,4,5,6,7,8], [8,7,6,5,4,3,2,1], 8)

In [ ]:
# каловое скрещивание
def two_point_crossover(parent1, parent2, chromo_length):
    chromo_length-=2
    par1_start=parent1[0]
    par2_start=parent2[0]
    par1_end=parent1[len(parent1)-1]
    par2_end=parent2[len(parent1)-1]
    parent1=parent1[1:len(parent1)-1]
    parent2=parent2[1:len(parent2)-1]

    pos1=random.randint(1, chromo_length//2-1)
    pos2=random.randint(pos1+1, chromo_length-1)

    child1=parent1[0:pos1]+parent2[pos1-1:pos2]+parent1[pos2+1:chromo_length]
    child2=parent2[0:pos1]+parent1[pos1-1:pos2]+parent2[pos2+1:chromo_length]

    child1.insert(0, par1_start)
    child1.append(par1_end)
    child2.insert(0, par2_start)
    child2.append(par2_end)
    return child1, child2

two_point_crossover([1,2,3,4,5,6,7,8], [8,7,6,5,4,3,2,1], 8)

In [ ]:
def mutation(chromosome, chrom_length):
    result=chromosome
    choice=random.random()
    
    if(choice-mutation_kek<=0):
        pos1=random.randint(2, chrom_length//2-1)
        pos2=random.randint(pos1+1, chrom_length-2)
        shrek=result[pos1:pos2+1]
        random.shuffle(shrek)
        result[pos1:pos2+1]=shrek

    return result

mutation([1,2,3,4,5,6,7,8], 8)